In [100]:
import json
import random
import ddddocr
import numpy as np
import cv2
import base64
from io import BytesIO

点字验证测试（知道为什么作者要屏蔽终端的输出了，因为会不可避免的输出一些奇怪的东西）

In [103]:
with open("./data/click_word.json", 'r', encoding="utf-8") as f:
    data = json.load(f)

In [119]:
def ocr_captcha(base64_img, word_list):
    det = ddddocr.DdddOcr(det=True)
    ocr = ddddocr.DdddOcr(beta=True)

    img = base64.b64decode(base64_img)
    stream = BytesIO(img)
    image_bytes = stream.read()

    poses = det.detection(image_bytes)
    print('poses: ', poses)

    arr = np.frombuffer(img, np.uint8)
    im = cv2.imdecode(arr, cv2.IMREAD_COLOR)

    decode_dict = {}
    for box in poses:
        x1, y1, x2, y2 = box
        cropped_img = im[y1:y2, x1:x2]
        cv2.imwrite("cropped.jpg", cropped_img)
        with open("cropped.jpg", 'rb') as f:
            cropped_img = f.read()
        res = ocr.classification(cropped_img)
        print(res)
        decode_dict[res] = {
            'x': int((x1 + x2) / 2),
            'y': int((y1 + y2) / 2),
        }
    print('decode_dict: ', decode_dict)

    res = []
    for word in word_list:
        if word in decode_dict.keys():
            res.append(decode_dict[word])
        else:
            candidates = list(filter(lambda x: x not in word_list, decode_dict.keys()))
            # 碰运气
            res.append(decode_dict[random.choice(candidates)])
    return res

In [120]:
base64_img = data['originalImageBase64']
word_list = data['wordList']
token = data['token']
point_arr = ocr_captcha(base64_img, word_list)

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
poses:  [[226, 89, 256, 119], [48, 19, 76, 46], [95, 85, 125, 115], [161, 71, 191, 101], [187, 61, 211, 88]]
生
几
直
怎
养
decode_dict:  {'生': {'x': 241, 'y': 104}, '几': {'x': 62, 'y': 32}, '直': {'x': 110, 'y': 100}, '怎': {'x': 176, 'y': 86}, '养': {'x': 199, 'y': 74}}


In [110]:
point_arr

[{'x': 241, 'y': 104}, {'x': 62, 'y': 32}, {'x': 110, 'y': 100}]

In [108]:
word_list

['生', '几', '直']

In [109]:
def decode_base64_to_image(base64_string, output_path):
    with open(output_path, "wb") as image_file:
        image_file.write(base64.b64decode(base64_string))

# 示例用法
output_path = "decoded_image.png"
decode_base64_to_image(base64_img, output_path)